In [24]:
import requests
import time
import json
from bs4 import BeautifulSoup


PTT_URL = 'https://www.ptt.cc'


def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'lxml')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').text.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)  # 轉換字串為數字
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2', ...
                    # 若不是, 不做任何事，push_count 保持為 0
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').text
                author = ''
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count,
                    'author': author
                })
    return articles, prev_url


if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Gossiping/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        today = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, today)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, today)

        # 儲存或處理文章資訊
        print('今天有', len(articles), '篇文章')
        threshold = 50
        print('熱門文章(> %d 推):' % (threshold))
        for a in articles:
            if int(a['push_count']) > threshold:
                print(a)
        with open('gossiping.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

今天有 1211 篇文章
熱門文章(> 50 推):
{'title': '[問卦] 如果有人對垃圾菌比中指？', 'href': '/bbs/Gossiping/M.1515214561.A.965.html', 'push_count': 65, 'author': ''}
{'title': 'Re: [爆卦] 幹！資進黨圍了史上最大的禁制區！', 'href': '/bbs/Gossiping/M.1515213091.A.8DC.html', 'push_count': 53, 'author': ''}
{'title': 'Re: [爆卦] 幹！資進黨圍了史上最大的禁制區！', 'href': '/bbs/Gossiping/M.1515212848.A.195.html', 'push_count': 68, 'author': ''}
{'title': '[問卦] 烏龍派出所哪一集最好笑', 'href': '/bbs/Gossiping/M.1515210828.A.8DF.html', 'push_count': 57, 'author': ''}
{'title': '[爆卦] 幹！資進黨圍了史上最大的禁制區！', 'href': '/bbs/Gossiping/M.1515211128.A.230.html', 'push_count': 99, 'author': ''}
{'title': '[新聞] 5時力立委轉戰凱道反勞基法修惡 黃國昌要', 'href': '/bbs/Gossiping/M.1515208693.A.6E2.html', 'push_count': 56, 'author': ''}
{'title': '[新聞] NET新年送暖 捐50萬提貨券嘉惠貧童', 'href': '/bbs/Gossiping/M.1515208731.A.630.html', 'push_count': 99, 'author': ''}
{'title': '[新聞] 大勝利！政大宣布廢除「英語畢業門檻」 學生歡呼：不用考多', 'href': '/bbs/Gossiping/M.1515202895.A.17E.html', 'push_count': 67, 'author': ''}
{'title': '[新聞] 警方舉